# Thực nghiệm thuật toán Gradient Boost trên tập dataset 1

<h2> Table of content </h2>
<div class ="alert alert-block alert-info" style="margin-top: 20px">
  <ol>
    <li><a href = "#1"> Tạo tập dữ liệu </a></li>
    <li><a href = "#2"> Thử nghiệm mô hình mặc định </a></li>
    <li><a href = "#tuning"> Tinh chỉnh siêu tham số  </a></li>
  </ol>
</div>

<h3 id = "1"> 1. Tạo tập dữ liệu </h3>

In [1]:
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
data_train = pd.read_csv("Data1_train.csv")
data_test = pd.read_csv("Data1_test.csv")
drop_columns = data_train.columns[:2]  #drop column "Unnamed", "ID"

data_train.drop(axis = 1, columns = drop_columns, inplace=True)
data_test.drop(axis = 1, columns = drop_columns, inplace=True)

data_train.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,4,2,3,5,178,3,1,0,7,4135,0
1,0,2,6,3,202,4,2,0,3,4496,1
2,3,2,2,5,277,3,2,1,6,1032,0
3,2,2,5,5,219,2,2,1,4,4289,0
4,2,0,6,3,296,4,1,0,4,1650,0


Dữ liệu cần dự đoán ở đây là cột Reach.on.Time_Y.N

In [3]:
X_train = data_train.iloc[:, data_train.columns != "Reached.on.Time_Y.N"].values
y_train = data_train["Reached.on.Time_Y.N"].values

X_test = data_test.iloc[:, data_test.columns != "Reached.on.Time_Y.N"].values
y_test = data_test["Reached.on.Time_Y.N"].values

<h3 id = "2"> 2. Thử nghiệm mô hình mặc định </h3>

In [4]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.58      0.87      0.69       895
           1       0.86      0.56      0.68      1305

    accuracy                           0.69      2200
   macro avg       0.72      0.72      0.69      2200
weighted avg       0.75      0.69      0.68      2200



<h3 id = "tuning"> 3. Tinh chỉnh siêu tham số </h3>


Các siêu tham số trong mô hình Gradient Boost bao gồm:

<li> <b> Learning rate </b>: Đây là một tham số điều chỉnh kích thước của bước cập nhật trong quá trình học tập. Một giá trị learning rate quá cao có thể dẫn đến việc mô hình bị quá khớp, trong khi một giá trị quá thấp có thể dẫn đến việc mô hình bị thiếu khớp.
<li> <b> Number of estimators </b>: Đây là số lượng cây quyết định được sử dụng trong mô hình. Một số lượng cây quyết định quá ít có thể dẫn đến việc mô hình không đủ tốt, trong khi một số lượng quá nhiều có thể dẫn đến việc mô hình bị quá khớp.
<li> <b> Maximum depth </b>: Đây là độ sâu tối đa của mỗi cây quyết định. Một độ sâu cây quá lớn có thể dẫn đến việc mô hình bị quá khớp, trong khi một độ sâu quá thấp có thể dẫn đến việc mô hình không đủ tốt.
<li> <b> Min samples split </b>: Đây là số lượng mẫu tối thiểu cần thiết để phân chia một nút trong cây quyết định. Một giá trị quá thấp có thể dẫn đến việc mô hình bị thiếu khớp, trong khi một giá trị quá cao có thể dẫn đến việc mô hình không đủ tốt

Siêu tham số được tinh chỉnh bằng phương pháp đánh giá chéo (cross validaition) với việc tập dữ liệu được chia làm 10 phần không chồng chéo và có kích thước bằng nhau. 
Ngoài ra thước đo cho mô hình em chọn là F1 - score

In [5]:
from sklearn.model_selection import GridSearchCV

model = GradientBoostingClassifier()
param_grid = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [60, 80, 100],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 3, 4],
}
grid_search = GridSearchCV(model, param_grid, cv=10, scoring= 'f1_micro')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'learning_rate': 0.1, 'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 60}


In [6]:
model = GradientBoostingClassifier(learning_rate= 0.1, max_depth= 3, min_samples_split=2, n_estimators= 60)
model.fit(X_train, y_train)
predict = model.predict(X_test)
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.58      0.92      0.71       895
           1       0.91      0.54      0.67      1305

    accuracy                           0.69      2200
   macro avg       0.74      0.73      0.69      2200
weighted avg       0.77      0.69      0.69      2200



In [ ]:
import os
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

if not os.path.exists('images'):
    os.makedirs('images')

for i in range(len(model.estimators_)):
    fig = plt.figure(figsize=(25,20))
    _ = plot_tree(model.estimators_[i][0], filled=True)
    fig.savefig(f"images/decistion_tree_{i+1}.png")
    plt.close()